# %% LE PLAN

'''
    1. create rations for each brain region: DOP/DAP, HIA/5HT, 3MT/DAP
    2. calculate and save to table average, SD and SEM
    3. ShapiroWilk test for normailty (note if not noirmal) and statistical analysis
    4. plot histograms with SEM and ** for each treatment, compound and BR
    5. test normaility for correlation if normal : 
                                                    --> pearson product coirrelation plots 
                            non-parametric data :
                                                    --> spearman coirrelation plots      
                                                    
            produced for each BR comparing compunds and compound comparing BR
    6. PCA for  complete compound_BR/ ratio_BR sets between groups 


'''

Imports and Constants:

In [1]:

from HPLC_MODULE_NOTEBOOK import * #Must be reimported when changes made to module file

filename = 'TCB2_data_APRIL23.csv'  # TCB2 #using current working directory plus file name 
compound_ratio_mapping = {'DOPAC': ['DA'],     # TCB2
                       '5HIAA': ['5HT'],
                       '3MT': ['DA'],
                       'HVA': ['DA', '3MT', 'DOPAC'],
                       'GLN': ['GLU']}

treatment_mapping = {1: 'vehicles', 2: '10mg/kgTCB', 3: '3mg/kgTCB',
                  4: '0.3mg/kgTCB', 5: 'TCB+MDL', 6: '0.2mg/kgMDL'} 

palette_labeled = {'vehicles': "white",  # TCB2
                   '10mg/kgTCB': "firebrick",
                   '3mg/kgTCB': "red",
                   '0.3mg/kgTCB': "salmon",
                   'TCB+MDL': "grey",
                   '0.2mg/kgMDL': "black"}

setTreatment(filename, treatment_mapping)
ratios_df = getRatiosDf(filename)
getRatiosPerRegion(filename, compound_ratio_mapping) #ratios of interest

# doGrubsTest(getRawDF()) #Usage is something like this


TREATMENT MAPPING {1: 'vehicles', 2: '10mg/kgTCB', 3: '3mg/kgTCB', 4: '0.3mg/kgTCB', 5: 'TCB+MDL', 6: '0.2mg/kgMDL'} SAVED TO /home/remi/jasmine/input/cache/TCB2_data_APRIL23 SUBCACHE
GETTING "ratios_df" FROM "TCB2_data_APRIL23" CACHE
GETTING "DOPAC:DA_5HIAA:5HT_3MT:DA_HVA:DA_3MT_DOPAC_GLN:GLU" FROM "TCB2_data_APRIL23" CACHE


,mouse_id,treatment,BR_1,compound_1,BR_2,compound_2,compounds,ratio
1009,2,vehicles,OF,DOPAC,OF,DA,DOPAC/DA,1.441471
1016,2,vehicles,OF,DOPAC,PL,DA,DOPAC/DA,3.853794
1025,2,vehicles,OF,DOPAC,CC,DA,DOPAC/DA,2.173147
1032,2,vehicles,OF,DOPAC,IC,DA,DOPAC/DA,7.261334
1042,2,vehicles,OF,DOPAC,M,DA,DOPAC/DA,1.459981
...,...,...,...,...,...,...,...,...
3995213,70,10mg/kgTCB,CE,HVA,SN,DOPAC,HVA/DOPAC,0.296836
3995221,70,10mg/kgTCB,CE,HVA,VTA,DOPAC,HVA/DOPAC,0.140351
3995231,70,10mg/kgTCB,CE,HVA,DR,DOPAC,HVA/DOPAC,1.021433
3995240,70,10mg/kgTCB,CE,HVA,MR,DOPAC,HVA/DOPAC,2.074015


In [2]:
getRatiosPerRegion(filename, compound_ratio_mapping) #ratios of interest

GETTING "DOPAC:DA_5HIAA:5HT_3MT:DA_HVA:DA_3MT_DOPAC_GLN:GLU" FROM "TCB2_data_APRIL23" CACHE


,mouse_id,treatment,BR_1,compound_1,BR_2,compound_2,compounds,ratio
1009,2,vehicles,OF,DOPAC,OF,DA,DOPAC/DA,1.441471
1016,2,vehicles,OF,DOPAC,PL,DA,DOPAC/DA,3.853794
1025,2,vehicles,OF,DOPAC,CC,DA,DOPAC/DA,2.173147
1032,2,vehicles,OF,DOPAC,IC,DA,DOPAC/DA,7.261334
1042,2,vehicles,OF,DOPAC,M,DA,DOPAC/DA,1.459981
...,...,...,...,...,...,...,...,...
3995213,70,10mg/kgTCB,CE,HVA,SN,DOPAC,HVA/DOPAC,0.296836
3995221,70,10mg/kgTCB,CE,HVA,VTA,DOPAC,HVA/DOPAC,0.140351
3995231,70,10mg/kgTCB,CE,HVA,DR,DOPAC,HVA/DOPAC,1.021433
3995240,70,10mg/kgTCB,CE,HVA,MR,DOPAC,HVA/DOPAC,2.074015


Load and Restructure df:

In [ ]:
df_raw = pd.read_csv(path, header=0)
df = df_raw.copy() #cworking df 
df = df.replace(np.nan, 0) # to set all 0 to Nan

raw_col_names = df_raw.columns.copy()
result_rows = []

for ind, row in df_raw.iterrows(): #loop for row get mouse and group
    
    mouse_id = row[0]
    treatment = row[1]

    for val, col_name in zip(row[2:], raw_col_names[2:]): #loop within row

        if val > 0:
            compound, BR = col_name.split('_')
            result_rows.append([mouse_id, treatment, BR, compound, val])

restructured_df = pd.DataFrame(result_rows, columns=col_names)
restructured_df

In [ ]:
df

In [ ]:
result_df = pd.DataFrame({'mouse_id':[], 'treatment':[], 'BR':[], 'compound':[]})

for raw_name in raw_col_names:
    compound, BR = col_name.split('_')

In [ ]:
restructured_df.dtypes

In [ ]:
# Inputs: 
compound_ratio_dict = {'DOPAC': ['DA'],     # TCB2
                       '5HIAA': ['5HT'],
                       '3MT': ['DA'],
                       'HVA': ['DA', '3MT', 'DOPAC'],
                       'GLN': ['GLU']}

treatment_dict = {1: 'vehicles', 2: '10mg/kgTCB', 3: '3mg/kgTCB',
                  4: '0.3mg/kgTCB', 5: 'TCB+MDL', 6: '0.2mg/kgMDL'} 

palette_labeled = {'vehicles': "white",  # TCB2
                   '10mg/kgTCB': "firebrick",
                   '3mg/kgTCB': "red",
                   '0.3mg/kgTCB': "salmon",
                   'TCB+MDL': "grey",
                   '0.2mg/kgMDL': "black"}

#intergrate 
restructured_df = applyMappingToColumn(restructured_df, 'treatment', treatment_dict )
restructured_df.to_pickle('input/restructured_df')
restructured_df



In [ ]:
print(restructured_df.BR.unique(), len(restructured_df.BR.unique()))
print(restructured_df.compound.unique(), len(restructured_df.compound.unique()))
print(restructured_df.mouse_id.unique(), len(restructured_df.mouse_id.unique()))

In [ ]:
getRatios(restructured_df)

In [ ]:
cleaned = pd.merge(left=restructured_df, right=restructured_df, on='mouse_id', suffixes=['_1', '_2']) #merge every BR/compound combo to every other for each mouse
cleaned['ratio'] = cleaned.apply(lambda x: x.ng_mg_1 / x.ng_mg_2, axis=1) #calculate the ratio
cleaned

In [ ]:
#exclusion of outliers Grubbs test 
#create ratios first as if there is a systematic error for a BR (run together) the ratio may still be good and there deserved to have a grubbs test for its own 
#treatment, compound, BR

#generate mean, sd and sem table for each 'data set' i.e. treatment, BR, compound

In [ ]:
#statistical testing
#remi: here i need to build in flexable stats structure

#dose responce data (single factor) --> one way ANOVA --> post hoc Tukey 
#agonist antagonist data (multifactor pathway) --> two way ANOVA --> one way ANOVA --> post hoc Tukey



#histograms with stats plotted (idealy chosing the level of stats i.e. significance in two way ANOVA or only if it passes all?)

In [ ]:
df = pd.DataFrame({'mouse_id': ['1', '1', '1', '1', '1', '1', '1', '1', '1'],
                   'BR': ['A', 'A', 'B', 'B', 'C', 'C', 'A', 'B', 'C'],
                   'compound': ['X', 'Y', 'X', 'Y', 'X', 'Y', 'Z', 'Z', 'Z'],
                   'value': [10, 20, 30, 40, 50, 60, 70, 80, 90]})
df

In [ ]:

merged = pd.merge(left=df, right=df, on='mouse_id', suffixes=['_1', '_2'])
merged
# merged[(merged.BR_1 < merged.BR_2) | (merged.compound_1 < merged.compound_2)]
# unique = merged.drop(merged[(merged.BR_1 == merged.BR_2) & (merged.compound_1 == merged.compound_2)].index)
# pd.merge(left=merged, right=merged, left_on=['mouse_id', 'BR_1', 'compound_1', 'BR_2', 'compound_2'], right_on=['mouse_id', 'BR_2', 'compound_2', 'BR_1', 'compound_1'], how='inner')

# unique

In [ ]:
os.path.isfile(f'{CACHE_DIR}/ratios_df.pkl')

In [ ]:
len(merged[(merged.BR_1 < merged.BR_2) | (merged.compound_1 < merged.compound_2) & ~((merged.BR_1 > merged.BR_2) & (merged.compound_1 < merged.compound_2))])